In [ ]:
import sys
print('google.colab' in sys.modules)
if 'google.colab' in sys.modules:
    !git clone https://github.com/theoliver7/uzh-aml.git
    %cd uzh-aml
    %load_ext tensorboard
    !pip install -r colab-requirements.txt
    


In [ ]:
import glob
import importlib
import os
import time

import numpy as np
import torch
import torch.nn.functional as F
from sklearn.model_selection import KFold
from torch.utils.data import random_split
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

# IMPORT THE MODEL TO BE TRAINED !!!
import models.hgp_sl_param.models as models

In [ ]:
# Refresh model so that code is updated directly
importlib.reload(models)

In [ ]:
args = {
    'device': "cuda",
    'seed': 777,
    'batch_size': 64,
    'lr': 0.0001,
    'weight_decay': 0.001,
    'epochs': 1000,
    'patience': 100,
    'nhid': 128,
    'pooling_ratio': 0.3,
    'dropout_ratio': 0.5,
    'sample_neighbor': True,
    'sparse_attention': True,
    'structure_learning': True,
    'negative_slope': 0.2,
    'lamb': 1.0,
    'dist': "man",
    'num_layers': 2,
    'layers_readout': [0, 1] # Assume we have 3 layers (so K = 3) and you want all 3 hidden representations to be part of the readout, you need to set this to [0, 1, 2]. If you only want the first and last layer, you need to set this to [0, 2]
}


In [ ]:
torch.set_printoptions(edgeitems=10)

if torch.cuda.is_available():
    args["device"] = 'cuda'
# SET THE RANDOM SEED
torch.manual_seed(args['seed'])

dataset = TUDataset(os.path.join('../../data', "DD"), name="DD", use_node_attr=True)
args["num_classes"] = dataset.num_classes
args["num_features"] = dataset.num_features
num_test = int(len(dataset) * 0.1)
num_train = len(dataset) - num_test
train_dataset, _ = torch.utils.data.random_split(dataset, [num_train, num_test])
kfold = KFold(n_splits=5, shuffle=True, random_state=args['seed'])

In [ ]:
def compute_test(model, loader):
    model.eval()
    correct = 0.0
    loss_test = 0.0
    for data in loader:
        data = data.to(args["device"])
        out = model(data)
        pred = out.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
        loss_test += F.nll_loss(out, data.y).item()
    return correct / len(loader.dataset), loss_test

In [ ]:
def train_fold(fold, train_idx, val_idx):
    train_subset = torch.utils.data.Subset(train_dataset, train_idx)
    val_subset = torch.utils.data.Subset(train_dataset, val_idx)
    
    train_loader = DataLoader(train_subset, batch_size=args["batch_size"], shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=args["batch_size"], shuffle=False)

    model = models.Model(args).to(args["device"])
 
    optimizer = torch.optim.Adam(model.parameters(), lr=args["lr"], weight_decay=args["weight_decay"])
    min_loss = 1e10
    patience_cnt = 0
    val_loss_values = []
    val_acc_values = []
    best_epoch = 0

    t = time.time()
    model.train()
    for epoch in range(args["epochs"]):
        loss_train = 0.0
        correct = 0
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            data = data.to(args["device"])
            out = model(data)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
            pred = out.max(dim=1)[1]
            correct += pred.eq(data.y).sum().item()
        acc_train = correct / len(train_loader.dataset)
        acc_val, loss_val = compute_test(model, val_loader)
        
        val_loss_values.append(loss_val)
        val_acc_values.append(acc_val)
        if val_loss_values[-1] < min_loss:
            min_loss = val_loss_values[-1]
            best_epoch = epoch
            patience_cnt = 0
        else:
            patience_cnt += 1

        if patience_cnt == args["patience"]:
            break

        
    print('Optimization Finished! Total time elapsed: {:.6f}'.format(time.time() - t))
    return val_acc_values[best_epoch], val_loss_values[best_epoch]

In [ ]:
import json

def select_layers_readout(layers,idx):
    if idx == 1:
        return [i for i in range(layers)]
    elif idx == 2:
        return [0]
    else:
        return [layers-1]

def save_to_json(data):
    if os.path.exists('cv_results.json'):
        with open('cv_results.json', 'r') as f:
            json_file_data = json.load(f)

        json_file_data.append(data)

        with open('cv_results.json', 'w') as f:
            json.dump(json_file_data, f, indent=3)
    else:
        with open('cv_results.json', 'w') as f:
            json.dump(data, f, indent=3)
            
layers = [2,3,4,5]
pool_ratio = [0.2,0.5,0.8]
node_inf_score_metric = ["man","euc"]
layers_readout = [1,2,3]
results = []
for l in layers:
    for p in pool_ratio:
        for s_m in node_inf_score_metric:
            for lr in layers_readout:
                lr = select_layers_readout(l,lr)
                args['num_layers'] = l
                args['pooling_ratio'] = p
                args['dist'] = s_m
                args['layers_readout'] = lr
                for fold, (train_idx, val_idx) in enumerate(kfold.split(np.arange(len(train_dataset)))):
                    val_acc, val_loss = train_fold(fold, train_idx, val_idx)
                    results.append((val_acc, val_loss))
                avg_fold_acc,avg_fold_loss = np.array(results).mean(axis=0)
                save_dict = [{'arguments':args,
                             'average fold accuracy':avg_fold_acc,
                             'average fold loss': avg_fold_loss}]
                save_to_json(save_dict)

                print(f"Cross-validation results: {avg_fold_acc} / {avg_fold_loss}, with the args {args}")

